In [11]:
import arcpy
from arcpy.sa import *
import datetime
from pysolar.solar import get_altitude, get_azimuth
import pytz

In [ ]:
# 设置工作空间和输入DEM
arcpy.env.workspace = r"C:\YourWorkspace"
arcpy.env.overwriteOutput = True
# 单线程
arcpy.env.parallelProcessingFactor = "0"

input_dem = "your_dem"

In [ ]:
Basin

In [ ]:
import arcpy
from arcpy.sa import *
import datetime
from pysolar.solar import get_altitude, get_azimuth
import pytz
# 设置工作空间和输入DEM
arcpy.env.workspace = r"C:\YourWorkspace"
arcpy.env.overwriteOutput = True
# 单线程
arcpy.env.parallelProcessingFactor = "0"

input_dem = "your_dem"
latitude = 	26.171786  # 示例：都匀的纬度
longitude = 107.482254  # 示例：都匀的经度
calculation_date = datetime.date(2023, 11, 22)  
def calculate_average_hillshade(dem, lat, lon, date):
    """
    计算指定位置和日期的8:00到16:00的平均山体阴影
    :param dem: 输入的DEM栅格
    :param lat: 纬度
    :param lon: 经度
    :param date: 日期 (datetime.date 对象)
    :return: 平均山体阴影栅格
    """
    # 设置时区
    timezone = pytz.timezone('Asia/Shanghai')  # 请根据您的位置调整时区
    
    hillshades = []
    
    for hour in range(8, 17):  # 8:00 to 16:00
        time = datetime.datetime.combine(date, datetime.time(hour, 0))
        time_utc = timezone.localize(time).astimezone(pytz.UTC)
        
        # 计算太阳高度角和方位角
        altitude = get_altitude(lat, lon, time_utc)
        azimuth = get_azimuth(lat, lon, time_utc)
        
        # 确保高度角为正（太阳在地平线上）
        if altitude > 0:
            # 计算山体阴影
            hillshade = Hillshade(dem, azimuth, altitude, "NO_SHADOWS", 1)
            hillshades.append(hillshade)
    
    # 计算平均山体阴影
    if hillshades:
        avg_hillshade = CellStatistics(hillshades, "MEAN")
        avg_hillshade.save("average_hillshade")
        return avg_hillshade
    else:
        print("警告：在指定的时间范围内没有有效的日照。")
        return None
def calculate_slope(dem):
    """计算坡度"""
    slope = slope_class(dem)
    slope.save("slope")
    return slope

def calculate_aspect(dem):
    """计算坡向"""
    aspect = Aspect(dem)
    aspect.save("aspect")
    return aspect

def calculate_plan_curvature(dem):
    """计算平面曲率"""
    plan_curve = Curvature(dem, "PLAN_CURVE")
    plan_curve.save("plan_curvature")
    return plan_curve

def calculate_profile_curvature(dem):
    """计算剖面曲率"""
    profile_curve = Curvature(dem, "PROFILE_CURVE")
    profile_curve.save("profile_curvature")
    return profile_curve

def calculate_convergence_index(dem):
    """计算汇聚指数"""
    aspect = calculate_aspect(dem)
    
    def relative_aspect(direction):
        diff = Abs(aspect - direction)
        return Con(diff > 180, 360 - diff, diff)
    
    rel_aspects = [relative_aspect(i * 45) for i in range(8)]
    avg_rel_aspect = CellStatistics(rel_aspects, "MEAN")
    convergence_index = 90 - avg_rel_aspect
    
    convergence_index.save("convergence_index")
    return convergence_index

def calculate_closed_depressions(dem):
    """计算封闭洼地"""
    filled_dem = Fill(dem)
    depressions = filled_dem - dem
    depressions.save("closed_depressions")
    return depressions

def calculate_catchment_area(dem):
    """计算汇水面积"""
    desc = Describe(dem)
    Cell_Size = desc.meanCellWidth
    flow_dir = FlowDirection(dem)
    flow_acc = FlowAccumulation(flow_dir)
    catchment_area = flow_acc * Cell_Size * Cell_Size
    catchment_area.save("catchment_area")
    return catchment_area

def calculate_wetness_index(dem):
    """计算湿度指数"""
    desc = Describe(dem)
    Cell_Size = desc.meanCellWidth
    slope = calculate_slope(dem)
    flow_acc = FlowAccumulation(FlowDirection(dem))
    wetness = Ln((flow_acc + 1) * Cell_Size / (Sin(slope * 0.01745) + 0.001))
    wetness.save("wetness_index")
    return wetness
import arcpy
from arcpy.sa import *

def calculate_twi_from_dem(dem_raster,transmissivity_raster=None, area_conversion=0, method=0):
    """
    根据 DEM 计算地形湿润指数 (TWI)
    Parameters:
    dem_raster (str): 输入 DEM 栅格路径
    output_twi_raster (str): 输出 TWI 栅格路径
    transmissivity_raster (str): 可选的传导率栅格路径，默认为 None
    area_conversion (int): 面积转换选项，0 表示不转换，1 表示转换，默认 0
    method (int): 计算方法，0 表示标准方法，1 表示 TOPMODEL 方法，默认 0
    """
    # 加载输入 DEM 栅格
    dem = Raster(dem_raster)
    # 计算坡度（以度为单位）
    slope = Slope(dem, "DEGREE")
    # 计算流向
    flow_dir = FlowDirection(dem)
    # 计算汇水面积
    sca = FlowAccumulation(flow_dir)
    # 计算 tan(Beta)
    tan_beta = Tan(slope)
    # 根据面积转换选项进行处理
    if area_conversion == 1:
        desc = arcpy.Describe(dem_raster)
        cell_size = desc.meanCellWidth
        sca = sca / cell_size
    # 根据计算方法和传导率选项计算 TWI
    if method == 0:  # 标准方法
        if transmissivity_raster:
            transmissivity = Raster(transmissivity_raster)
            twi = Ln(sca / (transmissivity * tan_beta))
        else:
            twi = Ln(sca / tan_beta)
    elif method == 1:  # TOPMODEL 方法
        if transmissivity_raster:
            transmissivity = Raster(transmissivity_raster)
            twi = Ln(sca / (transmissivity * tan_beta))
        else:
            twi = Ln(sca / tan_beta)
    # 保存结果
    twi.save("twi")
    return twi

def calculate_ls_factor(dem):
    """计算LS因子"""
    desc = Describe(dem)
    Cell_Size = desc.meanCellWidth
    slope = calculate_slope(dem,"DEGREE")
    flow_acc = FlowAccumulation(FlowDirection(dem))
    ls_factor = Pow(flow_acc * Cell_Size / 22.13, 0.6) * Pow(Sin(slope * 0.01745) / 0.0896, 1.3)
    ls_factor.save("ls_factor")
    return ls_factor

def calculate_channel_network(dem, threshold=1000):
    """计算通道网络"""
    flow_acc = FlowAccumulation(FlowDirection(dem))
    channel_network = Con(flow_acc > threshold, 1)
    channel_network.save("channel_network")
    return channel_network

def calculate_drainage_basins(dem):
    """计算流域"""
    flow_dir = FlowDirection(dem)
    basins = Basin(flow_dir)
    basins.save("drainage_basins")
    return basins

def calculate_channel_network_base_level(dem, channel_network):
    """计算通道网络基准面"""
    chnl_base = Con(IsNull(channel_network), dem)
    chnl_base.save("channel_network_base_level")
    return chnl_base

def calculate_channel_network_distance(channel_network):
    """计算到通道网络的距离"""
    chnl_dist = EucDistance(channel_network)
    chnl_dist.save("channel_network_distance")
    return chnl_dist

def calculate_valley_depth(dem, channel_network):
    """计算谷深"""
    chnl_base = calculate_channel_network_base_level(dem, channel_network)
    valley_depth = dem - chnl_base
    valley_depth.save("valley_depth")
    return valley_depth

def calculate_relative_slope_position(dem):
    """计算相对坡位"""
    neighborhood = NbrRectangle(5, 5, "CELL")
    min_elev = FocalStatistics(dem, neighborhood, "MINIMUM")
    max_elev = FocalStatistics(dem, neighborhood, "MAXIMUM")
    rel_slope_pos = (dem - min_elev) / (max_elev - min_elev)
    rel_slope_pos.save("relative_slope_position")
    return rel_slope_pos